## Setup

In [1]:
# Import Dependencies.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import requests
import json

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

from joblib import dump, load

In [2]:
# Fetch the data from the API.
listings_json = requests.get("http://127.0.0.1:5000/housingDataAPI/v1.0/listings").json()

# Examine the data.
print(json.dumps(listings_json[0], indent=4, sort_keys=True))

{
    "address": "255 SW HARRISON ST 19b, Portland OR 97201",
    "bathrooms": 1.0,
    "bedrooms": 1,
    "built": 1965,
    "city": "Portland",
    "county": "Multnomah",
    "elementary_school": "Ainsworth",
    "high_school": "Lincoln",
    "home_type": "Condo - Contemporary",
    "lot_size": null,
    "middle_school": "West Sylvan",
    "neighborhood": "HARRISON WEST PSU OHSU",
    "price": 250000,
    "square_feet": 744,
    "zipcode": 97201
}


In [3]:
# Create a dataframe to use for our model.
data_df = pd.DataFrame(listings_json)

print(len(data_df))
data_df.head()

1619


,address,bathrooms,bedrooms,built,city,county,elementary_school,high_school,home_type,lot_size,middle_school,neighborhood,price,square_feet,zipcode
0,"255 SW HARRISON ST 19b, Portland OR 97201",1.0,1,1965,Portland,Multnomah,Ainsworth,Lincoln,Condo - Contemporary,NaN,West Sylvan,HARRISON WEST PSU OHSU,250000,744,97201
1,"1811 NW COUCH ST 410, Portland OR 97209",1.0,1,1916,Portland,Multnomah,Chapman,Lincoln,Condo - Tudor,NaN,West Sylvan,unknown,250000,519,97209
2,"2211 SW PARK PL 102, Portland OR 97205",1.0,1,1965,Portland,Multnomah,Chapman,Lincoln,Condo - Contemporary,NaN,West Sylvan,KINGS HILL,250000,849,97205
3,"501 NE Bridgeton RD 5, Portland OR 97211",1.0,2,1964,Portland,Multnomah,Faubion,Jefferson,Floating Home - 1 Story,NaN,Other,BRIDGETON,250000,791,97211
4,"8712 N DECATUR ST 301, Portland OR 97203",1.0,2,2007,Portland,Multnomah,James John,Roosevelt,Condo - Contemporary,NaN,George,unknown,252000,934,97203


## Data Preprocessing

In [4]:
# Simplify home types 
for i in data_df.index:
    if "Floating" in data_df.at[i, "home_type"]:
        data_df.at[i, "home_type"] = "Floating"
    if "Condo" in data_df.at[i, "home_type"]:
        data_df.at[i, "home_type"] = "Condo"
    if "Single Family" in data_df.at[i, "home_type"]:
        data_df.at[i, "home_type"] = "Single Family"
    if "Manufactured" in data_df.at[i, "home_type"]:
        data_df.at[i, "home_type"] = "Manufactured"
    
data_df.home_type.unique() 

array(['Condo', 'Floating', 'Single Family'], dtype=object)

In [5]:
# Print data to compare how many data points lost
print(f'Current Amount of Listings: {len(data_df)}')

# Change lot size to 0 for floating homes and condos
for i in data_df.index:
    if data_df.at[i, "home_type"] == "Floating":
        data_df.at[i, "lot_size"] = 0
    if data_df.at[i, "home_type"] == "Condo":
        data_df.at[i, "lot_size"] = 0

# Drop listing with null lot_size
data_df.drop(data_df[data_df["lot_size"].isnull()].index, inplace=True)
      
# Print length of data
print(f'Updated Amount of Listings: {len(data_df)}')

Current Amount of Listings: 1619
Updated Amount of Listings: 1523


In [6]:
# Print data to compare how many data points lost
print(f'Current Amount of Listings: {len(data_df)}')

# Drop rows where high school cannot be compared.
data_df.drop(data_df[data_df.high_school == "Other"].index, inplace = True)

# Print length of data
print(f'Updated Amount of Listings: {len(data_df)}')

Current Amount of Listings: 1523
Updated Amount of Listings: 1521


In [7]:
# Create district df
school_dict = ({"high_school" : ['Reynolds', 'Parkrose', 'David Douglas', 'Centennial', 'Cleveland',
        'Lincoln', 'Madison', 'Jefferson', 'Roosevelt', 'Sunset','Westview', 'Liberty', 'Beaverton', 
        'Grant', 'Southridge', 'Tigard', 'Wilson', 'Riverdale', 'Lake Oswego', 'Franklin',
        'Tualatin', 'Milwaukie', 'Scappoose'], "district" : ['Reynolds', 'Parkrose','David Douglas',
        'Centennial', 'Portland Public', 'Portland Public', 'Portland Public', 'Portland Public',
        'Portland Public', 'Beaverton', 'Beaverton', 'Hillsboro', 'Beaverton', 'Portland Public',
        'Beaverton', 'Tigard-Tualatin', 'Portland Public', 'Riverdale', 'Lake Oswego', 'Portland Public',
        'Tigard-Tualatin', 'North Clackamas', 'Scappose']})
district_df = pd.DataFrame (school_dict)

# Merge into OG df
data_df = pd.merge(data_df, district_df, on="high_school")
data_df.head()

,address,bathrooms,bedrooms,built,city,county,elementary_school,high_school,home_type,lot_size,middle_school,neighborhood,price,square_feet,zipcode,district
0,"255 SW HARRISON ST 19b, Portland OR 97201",1.0,1,1965,Portland,Multnomah,Ainsworth,Lincoln,Condo,0.0,West Sylvan,HARRISON WEST PSU OHSU,250000,744,97201,Portland Public
1,"1811 NW COUCH ST 410, Portland OR 97209",1.0,1,1916,Portland,Multnomah,Chapman,Lincoln,Condo,0.0,West Sylvan,unknown,250000,519,97209,Portland Public
2,"2211 SW PARK PL 102, Portland OR 97205",1.0,1,1965,Portland,Multnomah,Chapman,Lincoln,Condo,0.0,West Sylvan,KINGS HILL,250000,849,97205,Portland Public
3,"111 SW HARRISON ST 7G, Portland OR 97201",1.0,1,1965,Portland,Multnomah,Ainsworth,Lincoln,Condo,0.0,West Sylvan,unknown,253000,600,97201,Portland Public
4,"1500 SW SKYLINE BLVD 16, Portland OR 97221",2.0,2,1966,Portland,Multnomah,Ainsworth,Lincoln,Condo,0.0,West Sylvan,Sylvan/Skyline,254900,1156,97221,Portland Public


In [8]:
# Add age of home column
data_df["house_age"] = 2020 - data_df["built"]
data_df.head()

,address,bathrooms,bedrooms,built,city,county,elementary_school,high_school,home_type,lot_size,middle_school,neighborhood,price,square_feet,zipcode,district,house_age
0,"255 SW HARRISON ST 19b, Portland OR 97201",1.0,1,1965,Portland,Multnomah,Ainsworth,Lincoln,Condo,0.0,West Sylvan,HARRISON WEST PSU OHSU,250000,744,97201,Portland Public,55
1,"1811 NW COUCH ST 410, Portland OR 97209",1.0,1,1916,Portland,Multnomah,Chapman,Lincoln,Condo,0.0,West Sylvan,unknown,250000,519,97209,Portland Public,104
2,"2211 SW PARK PL 102, Portland OR 97205",1.0,1,1965,Portland,Multnomah,Chapman,Lincoln,Condo,0.0,West Sylvan,KINGS HILL,250000,849,97205,Portland Public,55
3,"111 SW HARRISON ST 7G, Portland OR 97201",1.0,1,1965,Portland,Multnomah,Ainsworth,Lincoln,Condo,0.0,West Sylvan,unknown,253000,600,97201,Portland Public,55
4,"1500 SW SKYLINE BLVD 16, Portland OR 97221",2.0,2,1966,Portland,Multnomah,Ainsworth,Lincoln,Condo,0.0,West Sylvan,Sylvan/Skyline,254900,1156,97221,Portland Public,54


## Prepare Data for Model

In [9]:
# Include only those columns that will be used in the deep learning model.
model_df = data_df.loc[:, ["bathrooms",
                            "bedrooms",
                            "house_age",
                            "lot_size",
                            "square_feet",
                            "district",
                            "zipcode",
                            "price"]
                       ]

# Drop rows with NaN entries.
model_df.dropna(inplace=True)

# Check the model data.
print(len(model_df))
model_df.head()

1519


,bathrooms,bedrooms,house_age,lot_size,square_feet,district,zipcode,price
0,1.0,1,55,0.0,744,Portland Public,97201,250000
1,1.0,1,104,0.0,519,Portland Public,97209,250000
2,1.0,1,55,0.0,849,Portland Public,97205,250000
3,1.0,1,55,0.0,600,Portland Public,97201,253000
4,2.0,2,54,0.0,1156,Portland Public,97221,254900


In [10]:
# Analysis of price data.
housing_prices = model_df["price"]

quartiles = housing_prices.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of price is: {lowerq}")
print(f"The upper quartile of price is: {upperq}")
print(f"The interquartile range of price is: {iqr}")
print(f"The the median of price is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

outlier_model_df = model_df.loc[(model_df['price'] < lower_bound) 
                                        | (model_df['price'] > upper_bound)]
print(f"There are {len(outlier_model_df)} possible outliers.")

The lower quartile of price is: 399000.0
The upper quartile of price is: 799450.0
The interquartile range of price is: 400450.0
The the median of price is: 565000.0 
Values below -201675.0 could be outliers.
Values above 1400125.0 could be outliers.
There are 103 possible outliers.


In [11]:
# Bin prices into ten equal length ranges.
# print(pd.qcut(model_df["price"], 5, retbins=True))

# Define a function to place each listing into an interval up to the upper bound of prices.
def bin_price(price):
    """
    Input: Integer price of a home.
    Output: Pandas interval into which the price falls, outliers are put in the max bin up to the upper bound.
    """
    if price <= 375000:
        return pd.Interval(250000, 375000, closed="right")
    elif price <= 480000:
        return pd.Interval(375000, 480000, closed="right")
    elif price <= 650000:
        return pd.Interval(480000, 650000, closed="right")
    elif price <= 879900:
        return pd.Interval(650000, 879900, closed="right")
    else:
        return pd.Interval(879900, 1400125, closed="right")
    
# Add the price range column to the data frame.
model_df["price_range"] = model_df["price"].map(bin_price)
# Drop the original price data.
model_df.drop("price", axis=1, inplace=True)

model_df.head()

,bathrooms,bedrooms,house_age,lot_size,square_feet,district,zipcode,price_range
0,1.0,1,55,0.0,744,Portland Public,97201,"(250000, 375000]"
1,1.0,1,104,0.0,519,Portland Public,97209,"(250000, 375000]"
2,1.0,1,55,0.0,849,Portland Public,97205,"(250000, 375000]"
3,1.0,1,55,0.0,600,Portland Public,97201,"(250000, 375000]"
4,2.0,2,54,0.0,1156,Portland Public,97221,"(250000, 375000]"


In [12]:
# Get dummies for the values in home_type to use in the model.
model_df = pd.get_dummies(model_df, columns=["district"])
model_df.columns

Index(['bathrooms', 'bedrooms', 'house_age', 'lot_size', 'square_feet',
       'zipcode', 'price_range', 'district_Beaverton', 'district_Centennial',
       'district_David Douglas', 'district_Hillsboro', 'district_Lake Oswego',
       'district_North Clackamas', 'district_Parkrose',
       'district_Portland Public', 'district_Reynolds', 'district_Riverdale',
       'district_Scappose', 'district_Tigard-Tualatin'],
      dtype='object')

In [13]:
# Assign X (input) and y (target).

X = model_df.drop("price_range", axis=1)
y = model_df.loc[:,"price_range"].values.reshape(-1, 1)

In [14]:
# Split the data into training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [15]:
# Create a StandardScaler model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

# Save the scalar.
dump(X_scaler, 'standard_scaler.bin', compress=True)

['standard_scaler.bin']

In [16]:
# Transform the training and testing data using the X_scaler.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Label encode the target data.
label_encoder = LabelEncoder()
label_encoder.fit(np.ravel(y_train))
encoded_y_train = label_encoder.transform(np.ravel(y_train))
encoded_y_test = label_encoder.transform(np.ravel(y_test))

# Save the label encoder
print(y_train[0])
print(encoded_y_train[0])
dump(label_encoder, 'label_encoder.bin', compress=True)

[Interval(879900, 1400125, closed='right')]
4


['label_encoder.bin']

## Model Creation, Training, and Testing

In [18]:
# Create an mlp model.
model = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42)

In [19]:
# Use grid search to tune the model.
grid = GridSearchCV(model,
                    {
                        "alpha":10.0 ** -np.arange(1, 7),
                        "learning_rate_init":10.0 ** -np.arange(1, 7),
                        "max_iter":[100, 200, 300],
                        
                    },
                    verbose=2)
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] alpha=0.1, learning_rate_init=0.1, max_iter=100 .................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .. alpha=0.1, learning_rate_init=0.1, max_iter=100, total=   0.5s
[CV] alpha=0.1, learning_rate_init=0.1, max_iter=100 .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] .. alpha=0.1, learning_rate_init=0.1, max_iter=100, total=   0.8s
[CV] alpha=0.1, learning_rate_init=0.1, max_iter=100 .................
[CV] .. alpha=0.1, learning_rate_init=0.1, max_iter=100, total=   0.7s
[CV] alpha=0.1, learning_rate_init=0.1, max_iter=100 .................
[CV] .. alpha=0.1, learning_rate_init=0.1, max_iter=100, total=   0.5s
[CV] alpha=0.1, learning_rate_init=0.1, max_iter=100 .................
[CV] .. alpha=0.1, learning_rate_init=0.1, max_iter=100, total=   0.8s
[CV] alpha=0.1, learning_rate_init=0.1, max_iter=200 .................
[CV] .. alpha=0.1, learning_rate_init=0.1, max_iter=200, total=   0.5s
[CV] alpha=0.1, learning_rate_init=0.1, max_iter=200 .................
[CV] .. alpha=0.1, learning_rate_init=0.1, max_iter=200, total=   0.8s
[CV] alpha=0.1, learning_rate_init=0.1, max_iter=200 .................
[CV] .. alpha=0.1, learning_rate_init=0.1, max_iter=200, total=   0.9s
[CV] alpha=0.1, learning_rate_init=0.1, max_iter=200 .................
[CV] .

...

[CV]  alpha=1e-06, learning_rate_init=1e-06, max_iter=300, total=   5.0s
[CV] alpha=1e-06, learning_rate_init=1e-06, max_iter=300 .............


C:\Users\Tristan\anaconda\Anaconda3\envs\pythondata\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done 540 out of 540 | elapsed: 22.8min finished


[CV]  alpha=1e-06, learning_rate_init=1e-06, max_iter=300, total=   4.6s


C:\Users\Tristan\anaconda\Anaconda3\envs\pythondata\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=None, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08,
                                     hidden_layer_sizes=(100, 100),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random...
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
   

In [20]:
# List the best parameters and best score.
print(grid.best_params_)
print(grid.best_score_)

{'alpha': 1e-05, 'learning_rate_init': 0.001, 'max_iter': 300}
0.5943735991962285


## Quantify the trained model

In [21]:
model_accuracy = grid.score(X=X_test_scaled, y=encoded_y_test)
print(f"Accuracy: {model_accuracy}")

Accuracy: 0.6026315789473684


## Make Predictions

In [22]:
# Use the first 10 test data values to make a prediction and compare it to the actual labels.
encoded_predictions = grid.predict(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: [Interval(650000, 879900, closed='right')
 Interval(250000, 375000, closed='right')
 Interval(650000, 879900, closed='right')
 Interval(375000, 480000, closed='right')
 Interval(250000, 375000, closed='right')
 Interval(375000, 480000, closed='right')
 Interval(879900, 1400125, closed='right')
 Interval(375000, 480000, closed='right')
 Interval(480000, 650000, closed='right')
 Interval(650000, 879900, closed='right')]
Actual Labels: [array([Interval(650000, 879900, closed='right')], dtype=object), array([Interval(375000, 480000, closed='right')], dtype=object), array([Interval(879900, 1400125, closed='right')], dtype=object), array([Interval(480000, 650000, closed='right')], dtype=object), array([Interval(250000, 375000, closed='right')], dtype=object), array([Interval(250000, 375000, closed='right')], dtype=object), array([Interval(879900, 1400125, closed='right')], dtype=object), array([Interval(480000, 650000, closed='right')], dtype=object), array([Interval(65000

## Save the Trained Model

In [23]:
# Save the model
dump(grid, 'mlp_classifier.bin', compress=True)

['mlp_classifier.bin']

## Test the Saved Model, Scaler, and Label Encoder

In [24]:
# Load the model, scaler and label encoder.
model = load("mlp_classifier.bin")
scaler = load("standard_scaler.bin")
label_encoder = load("label_encoder.bin")

In [25]:
# Input data for testing.
input_data = np.array(np.array([X.iloc[0]]))

In [26]:
X.iloc[0]

bathrooms                       1.0
bedrooms                        1.0
house_age                      55.0
lot_size                        0.0
square_feet                   744.0
zipcode                     97201.0
district_Beaverton              0.0
district_Centennial             0.0
district_David Douglas          0.0
district_Hillsboro              0.0
district_Lake Oswego            0.0
district_North Clackamas        0.0
district_Parkrose               0.0
district_Portland Public        1.0
district_Reynolds               0.0
district_Riverdale              0.0
district_Scappose               0.0
district_Tigard-Tualatin        0.0
Name: 0, dtype: float64

In [27]:
encoded_predictions = model.predict(scaler.transform(input_data))
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

print(f"{prediction_labels[0].left}, {prediction_labels[0].right}")

250000, 375000
